In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
#df = pd.read_csv('../data_engineering/datasets/DOH Data Drop/08062022/DOH COVID Data Drop_ 20220806 - 04 Case Information_batch_3.csv')
df = pd.read_csv('../data_engineering/datasets/DOH Data Drop/09032022/DOH COVID Data Drop_ 20220903 - 04 Case Information_batch_3.csv')

df.head()

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,C85479845,22.0,20 to 24,FEMALE,2022-01-08,2022-01-09,2022-01-11,NaN,2022-01-10,RECOVERED,...,SURIGAO DEL SUR,CANTILAN,PH166805000,LININTI-AN (POB.),PH166805009,RECOVERED,NO,2022-01-01,NO,NaN
1,C54690312,36.0,35 to 39,MALE,2022-01-11,NaN,2022-01-11,NaN,NaN,RECOVERED,...,NCR,QUEZON CITY,PH137404000,TANDANG SORA,PH137404120,RECOVERED,NO,NaN,NaN,"Health Status is ""Recovered"", but no Date Reco..."
2,C46949462,28.0,25 to 29,MALE,2022-01-06,2022-01-08,2022-01-11,NaN,NaN,RECOVERED,...,CAVITE,BACOOR CITY,PH042103000,MABOLO II,PH042103037,RECOVERED,NO,NaN,NaN,"Health Status is ""Recovered"", but no Date Reco..."
3,C81759368,55.0,55 to 59,FEMALE,2022-01-07,2022-01-10,2022-01-11,NaN,2022-01-17,RECOVERED,...,NCR,PATEROS,PH137606000,SANTO ROSARIO-SILANGAN,PH137606009,RECOVERED,NO,NaN,NO,NaN
4,C59214277,52.0,50 to 54,MALE,2022-01-07,2022-01-08,2022-01-11,NaN,NaN,RECOVERED,...,RIZAL,CITY OF ANTIPOLO (CAPITAL),PH045802000,MUNTINGDILAW,PH045802018,RECOVERED,NO,NaN,NaN,"Health Status is ""Recovered"", but no Date Reco..."


In [3]:
df[df.HealthStatus != 'RECOVERED'].isna().sum()

CaseCode                 0
Age                    639
AgeGroup               639
Sex                      0
DateSpecimen          1830
DateResultRelease     1718
DateRepConf              0
DateDied             23644
DateRecover          28936
RemovalType          23571
Admitted               799
RegionRes               42
ProvRes                191
CityMunRes             240
CityMuniPSGC           236
BarangayRes            878
BarangayPSGC           874
HealthStatus             0
Quarantined              0
DateOnset            18627
Pregnanttab          13323
ValidationStatus     24241
dtype: int64

In [4]:
len(df[df.HealthStatus != 'RECOVERED'])

29148

In [5]:
#df_latest = df[df.DateRepConf == '2022-08-06']

df.drop(['CaseCode', 'Sex', 'Admitted', 'Quarantined', 'CityMunRes', 'Pregnanttab', 'DateOnset'], axis=1, inplace=True)

df.head()

,Age,AgeGroup,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,RegionRes,ProvRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,ValidationStatus
0,22.0,20 to 24,2022-01-08,2022-01-09,2022-01-11,NaN,2022-01-10,RECOVERED,CARAGA,SURIGAO DEL SUR,PH166805000,LININTI-AN (POB.),PH166805009,RECOVERED,NaN
1,36.0,35 to 39,2022-01-11,NaN,2022-01-11,NaN,NaN,RECOVERED,NCR,NCR,PH137404000,TANDANG SORA,PH137404120,RECOVERED,"Health Status is ""Recovered"", but no Date Reco..."
2,28.0,25 to 29,2022-01-06,2022-01-08,2022-01-11,NaN,NaN,RECOVERED,Region IV-A: CALABARZON,CAVITE,PH042103000,MABOLO II,PH042103037,RECOVERED,"Health Status is ""Recovered"", but no Date Reco..."
3,55.0,55 to 59,2022-01-07,2022-01-10,2022-01-11,NaN,2022-01-17,RECOVERED,NCR,NCR,PH137606000,SANTO ROSARIO-SILANGAN,PH137606009,RECOVERED,NaN
4,52.0,50 to 54,2022-01-07,2022-01-08,2022-01-11,NaN,NaN,RECOVERED,Region IV-A: CALABARZON,RIZAL,PH045802000,MUNTINGDILAW,PH045802018,RECOVERED,"Health Status is ""Recovered"", but no Date Reco..."


In [6]:
df_active_cases = df[df.HealthStatus != 'RECOVERED'].groupby('BarangayPSGC').agg(active_cases = ('BarangayPSGC', 'count')).sort_values('active_cases', ascending=False)

df_active_cases.head()

,active_cases
BarangayPSGC,
PH137607020,153
PH137605076,105
PH137604007,94
PH137603001,92
PH137403015,86


In [7]:
#gdf = gpd.read_file('../data_engineering/datasets/admin_bdries_lvl3/phl_admbnda_adm3_psa_namria_20200529.shp')
#gdf.drop(['Shape_Leng', 'Shape_Area', 'ADM3ALT1EN', 'ADM3ALT2EN', 'date', 'validOn', 'validTo'], axis=1, inplace=True)

gdf = gpd.read_file('E:\\Projects\\ManilaCOVID19Cases\\admin_boundaries\\Barangays.shp')

gdf.head()

,ADM1_PCODE,ADM2_PCODE,ADM3_PCODE,ADM4_PCODE,ADM4_EN,ADM3_EN,ADM2_EN,ADM1_EN,ADM_ID,UPDATED,PERIMETER,AREA,LONGITUDE,LATITUDE,geometry
0,PH010000000,PH015500000,PH015502000,PH015502012,Pogomboa,Aguilar,Pangasinan,Region I,15502012.0,2017-12-31,5590.506,1.745360e+06,120.243,15.882,"POLYGON ((120.25441 15.88112, 120.24723 15.878..."
1,PH010000000,PH015500000,PH015502000,PH015502013,Pogonsili,Aguilar,Pangasinan,Region I,15502013.0,2017-12-31,30658.516,2.011989e+07,120.215,15.791,"POLYGON ((120.26372 15.81724, 120.25821 15.811..."
2,PH010000000,PH015500000,PH015502000,PH015502014,San Jose,Aguilar,Pangasinan,Region I,15502014.0,2017-12-31,13404.869,5.289032e+06,120.280,15.867,"POLYGON ((120.28018 15.87048, 120.27885 15.870..."
3,PH010000000,PH015500000,PH015502000,PH015502016,Tampac,Aguilar,Pangasinan,Region I,15502016.0,2017-12-31,6235.011,1.721892e+06,120.251,15.875,"POLYGON ((120.26163 15.87600, 120.26149 15.875..."
4,PH010000000,PH015500000,PH015502000,PH015502017,Laoag,Aguilar,Pangasinan,Region I,15502017.0,2017-12-31,36419.270,3.944797e+07,120.191,15.856,"POLYGON ((120.22100 15.88999, 120.22163 15.886..."


In [8]:
df_active_cases = gpd.GeoDataFrame(df_active_cases)

In [9]:
merged = gpd.GeoDataFrame.merge(df_active_cases, gdf, how='right', left_on = 'BarangayPSGC', right_on='ADM4_PCODE')

merged.set_crs('epsg:4326')

merged.fillna(0, inplace=True)

#merged.to_file('../data_engineering/datasets/output_shps/active_cases_08062022/08062022.shp')
#merged.to_file('../data_engineering/datasets/output_shps/active_cases_09032022/09032022.shp')

In [10]:
#gpd.read_file('E:\\Projects\\ManilaCOVID19Cases\\admin_boundaries\\Barangays.shp')

In [11]:
merged[['active_cases', 'ADM4_EN', 'ADM3_EN']].sort_values('active_cases', ascending=False)[0:10]

,active_cases,ADM4_EN,ADM3_EN
34390,153.0,Fort Bonifacio,Taguig City
34226,105.0,Barangay 76,Pasay City
34141,94.0,B. F. Homes,City of Parañaque
34126,92.0,Alabang,City of Muntinlupa
33639,86.0,Pinagbuhatan,City of Pasig
33793,83.0,Batasan Hills,Quezon City
33608,81.0,Wack-wack Greenhills,City of Mandaluyong
30591,72.0,Buhangin (Pob.),Davao City
12534,71.0,Mayamot,City of Antipolo
34142,70.0,Don Bosco,City of Parañaque


In [12]:
merged.head()

,active_cases,ADM1_PCODE,ADM2_PCODE,ADM3_PCODE,ADM4_PCODE,ADM4_EN,ADM3_EN,ADM2_EN,ADM1_EN,ADM_ID,UPDATED,PERIMETER,AREA,LONGITUDE,LATITUDE,geometry
0,0.0,PH010000000,PH015500000,PH015502000,PH015502012,Pogomboa,Aguilar,Pangasinan,Region I,15502012.0,2017-12-31,5590.506,1.745360e+06,120.243,15.882,"POLYGON ((120.25441 15.88112, 120.24723 15.878..."
1,1.0,PH010000000,PH015500000,PH015502000,PH015502013,Pogonsili,Aguilar,Pangasinan,Region I,15502013.0,2017-12-31,30658.516,2.011989e+07,120.215,15.791,"POLYGON ((120.26372 15.81724, 120.25821 15.811..."
2,1.0,PH010000000,PH015500000,PH015502000,PH015502014,San Jose,Aguilar,Pangasinan,Region I,15502014.0,2017-12-31,13404.869,5.289032e+06,120.280,15.867,"POLYGON ((120.28018 15.87048, 120.27885 15.870..."
3,0.0,PH010000000,PH015500000,PH015502000,PH015502016,Tampac,Aguilar,Pangasinan,Region I,15502016.0,2017-12-31,6235.011,1.721892e+06,120.251,15.875,"POLYGON ((120.26163 15.87600, 120.26149 15.875..."
4,0.0,PH010000000,PH015500000,PH015502000,PH015502017,Laoag,Aguilar,Pangasinan,Region I,15502017.0,2017-12-31,36419.270,3.944797e+07,120.191,15.856,"POLYGON ((120.22100 15.88999, 120.22163 15.886..."
